In [ ]:
import os
import json
import pandas as pd
from tqdm import tqdm


from utils.prompts import generate_question_evaluation_prompt_v2
from utils.bedrock_functions import invoke_bedrock_endpoint, build_anthropic_request_body
from utils.bedrock_functions import build_mistral_request_body, build_nova_request_body, build_llama_request_body

from utils.helper_functions import read_jsonl_file, save_as_jsonl


In [ ]:
directory = "/home/ec2-user/preetam_experiments"
split = 'test'
data = json.load(open(f"{directory}/batch_processing_files/{split}/intermediate_res/qa_pairs_{split}.json"))